In [1]:
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

In [30]:
# Read in the three weather data files from 1-Raw_data
# Note that the second row contains the header names in the csv files

weather_data_2020 =  pd.read_csv(r"C:\Users\user\PycharmProjects\CIV1498Project2\0-Raw_data\weather_data_2019.csv", header=[1])
weather_data_2020 =  pd.read_csv(r"C:\Users\user\PycharmProjects\CIV1498Project2\0-Raw_data\weather_data_2020.csv", header=[1])
weather_data_2021 =  pd.read_csv(r"C:\Users\user\PycharmProjects\CIV1498Project2\0-Raw_data\weather_data_2021.csv", header=[1])

weather_data_2021.tail(8)

,Date,Time,TEMP_MEAN,WDIR_VECT,WDIR_UVEC,WSPD_SCLR,HUMIDITY
8785,Minimum,NaN,-39.3,0,0,0,10.9
8786,Min Date,NaN,2/7/2021 4:00,5/18/2021 13:00,10/5/2021 23:00,9/18/2021 5:00,5/21/2021 17:00
8787,Maximum,NaN,37.9,360,360,15.7,100
8788,Max Date,NaN,6/29/2021 17:00,4/17/2021 19:00,7/4/2021 16:00,2/21/2021 9:00,8/3/2021 3:00
8789,Avg,NaN,2.6,233.7,234.1,2.9,64.6
8790,Num,NaN,8778,8777,8777,8777,8778
8791,Data[%],NaN,100,100,100,100,100
8792,STD,NaN,13.3,85.4,86.7,2.6,19
